In [11]:
%pip install langchain-core langgraph>0.2.27
%pip install -qU langchain-google-vertexai
%pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [13]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatVertexAI(model = "gemini-1.5-pro", temperature = 0)

# system_message_backup = """
# - Input: "Jarabe para la tos seca apto para diabéticos."
#   Respuesta esperada: "Jarabe"
# - Input: "Una pasta de dientes para niños alérgicos al flúor"
#   Respuesta esperada: "Pasta de dientes"
# - Input: "Una crema hidratante con veneno de serpiente"
#   Respuesta esperada: "Crema"
# - Input: "Piruletas para la gargante"
#   Respuesta esperada: "Piruletas"
# """

system_message = """
Eres la primera capa de un chatbot que ayuda a farmacéuticos a identificar productos que solicitan clientes que vienen a la farmacia. 
Tu objetivo es solamente identificar la categoría del producto que se está demandando. Debes elegir entre las siguientes categorias:
- Ortopedia: Productos de ortopedia como bastones, muletas, conteras, empuñadoras, fajas lumbares y fajas en general, collarines y cosas del estilo que se acoplen a partes del cuerpo humano o a utensilios que estos puedan utilizar.
- Ojos: Productos para ojos como colirio, gafas presbicia, gafas solares, soluciones lentes, limpieza ocular, parches oculares, otros óptica, audiología.
Ejemplos:
- Input: Andador regulable para persona mayor con problemas de cadera
  Respuesta esperada: Ortopedia
- Input: Collarin para lesión en el cuello
  Respuesta esperada: Ortopedia
- Input: Coliria para ojos secos
  Respuesta esperada: Ojos
- Input: Producto para post operación ocular
  Respuesta esperada: Ojos 
"""

prompt = ChatPromptTemplate.from_messages(    
    [
    SystemMessage(content=system_message),
    MessagesPlaceholder(variable_name="messages") # Para mantener historial de conversación usamos MessagesPlaceholder
    ]
)


ValidationError: 1 validation error for ChatVertexAI
  Value error, Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project [type=value_error, input_value={'temperature': 0, 'model... 'default_metadata': ()}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    chain = prompt | llm
    response = chain.invoke(state)
    return {"messages": response}
    

workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer = memory)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

config = {"configurable": {"thread_id": "abcds12345"}}

query = "Necesito una colirio para sequedad ocular"

input = [HumanMessage(content=query)]

output = app.invoke({"messages": input}, config)

# print(output.messages.HumaMessage.content)
print(output)


NameError: name 'HumanMessage' is not defined